In [7]:
# いつも使うやつ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# pandasで全ての列を表示
pd.options.display.max_columns = None

# 図をipython notebook内で表示
%matplotlib inline

# DeplicatedWarningを避けるため
import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)

import sys
sys.path.append('./lib/')

# 交差検定をサクッとやるためのモジュールを読み込む
# from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score

# 説明変数つくるための関数
import feature_process_helper

In [8]:
# データをそれぞれ読み込む
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
submission_df = pd.read_csv('./data/gender_submission.csv')

In [12]:
train = train_df.copy()
test = test_df.copy()
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
train, test = feature_process_helper.names(train, test)
train, test = feature_process_helper.age_impute(train, test)
train, test = feature_process_helper.cabin_num(train, test)
train, test = feature_process_helper.cabin(train, test)
train, test = feature_process_helper.embarked_impute(train, test)
train, test = feature_process_helper.fam_size(train, test)
train['Ticket_Len'] = train['Ticket'].apply(lambda x: len(x))
test['Ticket_Len'] = test['Ticket'].apply(lambda x: len(x))
train, test = feature_process_helper.dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = feature_process_helper.drop(train, test, bye = ['Ticket'])

TypeError: names() missing 1 required positional argument: 'submission_df'

In [ ]:
# 分類モデル（他にもいろいろある）
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [ ]:
classifiers = [
    ('lr', LogisticRegression()), 
    ('knn', KNeighborsClassifier()),
    ('linear svc', SVC(kernel="linear")), # データ点が多いと計算に時間がかかる
    ('rbf svc', SVC(gamma=2)), # データ点が多いと計算に時間がかかる
    ('dt', DecisionTreeClassifier()),
    ('rf', RandomForestClassifier(random_state=42)),
    ('et', ExtraTreesClassifier()),
    ('ab', AdaBoostClassifier()),
    ('gbc', GradientBoostingClassifier()),
    ('gnb', GaussianNB()),
    ('lda', LinearDiscriminantAnalysis()),
    ('qda', QuadraticDiscriminantAnalysis()),
    #('xgb', XGBClassifier()) # これだけ個別にインストールする必要があるので今回は割愛
]

In [13]:
warnings.filterwarnings("ignore", category=UserWarning)

import time
results = {}
exec_times = {}

for name, model in classifiers:
    tic = time.time()
    if name in ['linear svc', 'rbf svc']:
        result = cross_val_score(model, train.iloc[:, 2:], train.iloc[:, 1], cv=5, scoring='accuracy')        
    else:
        result = cross_val_score(model, train.iloc[:, 2:], train.iloc[:, 1], cv=5, scoring='log_loss')
    exec_time = time.time() - tic
    exec_times[name] = exec_time
    results[name] = result
    
    print("{0:.3f} ({1:.3f}): time {2:.2f}s, {3}".format(result.mean(), result.std(), exec_time, name))

NameError: name 'classifiers' is not defined

In [ ]:
pd.DataFrame(results).iloc[0, :].sort_values(ascending=False)

SVC(kernel="linear")が一番大きいという認識でおｋ？

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 結果の描画を楽にするためpandasのデータフレームに結果を入れる
import pandas as pd

dfr = pd.DataFrame(results)

dfr[dfr.median().sort_values(ascending=True).index].boxplot(vert=False);

見方がわからん

### SVM('linear')

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['accuracy', 'precision', 'recall']

In [ ]:
from sklearn.grid_search import GridSearchCV

### 試し学習

In [ ]:
SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
svc.fit(train.iloc[:, 2:], train.iloc[:, 1])

In [ ]:
print(svc.score(train.iloc[:, 2:], train.iloc[:, 1]))

In [ ]:
result = cross_val_score(model, train.iloc[:, 2:], train.iloc[:, 1], cv=5, scoring='accuracy') 

In [ ]:
print("{0:.3f} ({1:.3f})".format(result.mean(), result.std()))

In [ ]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
predictions = svc.predict(test.iloc[:, 1:])
predictions = pd.DataFrame(predictions, columns=['Survived'])
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv('y_svc_linear.csv', sep=",", index = False)

kaggleにアップしたら  
0.77990 落ちた